In [ ]:
import torch

class SAO(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, alpha=0.05, **kwargs):
        """
        Инициализирует Sharpness-Aware Optimization (SAO) оптимизатор, минимизирует резкость функции потерь.

        Args:
            params (iterable): Параметры модели, которые обновляются.
            base_optimizer (torch.optim.Optimizer): Базовый оптимизатор, на который накладывается SAO. Возьмем AdamW.
            alpha (float, optional): Коэффициент, определяющий степень адаптивного сглаживания резкости. По умолчанию 0.05.
            **kwargs: Доп. аргументы для базового оптимизатора.
        """
        defaults = dict(alpha=alpha, **kwargs)  # значение гиперпараметров, словарь
        super(SAO, self).__init__(params, defaults)

        # Создание базового оптимизатора, чтобы выполнять обновления
        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups
        self.alpha = alpha  # коэффициент сглаживания


    def _calculate_grad_norm(self):
        """Вычисляет норму градиента для адаптивного сглаживания"""

        available_device = self.param_groups[0]["params"][0].device
        norm = torch.norm(
            torch.stack([
                p.grad.norm(p=2).to(available_device)  # p=2 для вычисления L2
                for group in self.param_groups for p in group["params"]
                if p.grad is not None
            ]),
            p=2
        )
        return norm


    @torch.no_grad()
    def step(self, closure=None):
        """Выполняет один шаг (step) оптимизации SAO.
        - grad_norm рассчитывает евклидову норму L2 всех градиентов для параметров модели, чтобы понять силу (вес) градиента.
            Используется для адаптивного изменения масштаба искажения.
        Чтобы изьежать деление на 0 => добавляем малое значение 1e-12

        Args:
            closure (_type_, optional): Проводит forward и backward прокоды по сети и вычисляет градиенты для каждого параметра.
        """

        assert closure is not None, "Closure is None. SAO cannot calculate gradients."

        # Первый проход: вычисляем градиенты на текущем значении параметров
        closure()

        # Рассчитываем адаптивное искажение на основе нормы градиентов
        grad_norm = self._calculate_grad_norm()
        for group in self.param_groups:
            scale = group["alpha"] / (grad_norm + 1e-12)  # масштабирование с адаптивной нормой
            for p in group["params"]:
                if p.grad is not None:  # если градиента у параметра p нет
                    # Искажение параметров с учетом резкости
                    p.data.add_(p.grad, alpha=scale) # градиенты сохраняются в p.grad для каждого p

        # Выполняем обновление с учетом адаптивного сглаживания
        self.base_optimizer.step()  # вызов базового оптимизатора
        self.zero_grad()  # очищаем градиенты, т.к. итерируется по батчам

In [ ]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 44.4 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""hw_3_2_pytorch.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1hQan7oAWLcAqYf-Va--JqzY5sHabu292

# Домашнее задание 2.2

В этом задании нужно:
1. Написать свою сеть на Pytorch по варианту
2. Обучить ее и сравнить результаты с дообученной сетью из зоопарка моделей
3. Поставить ряд экспериментов, показывающих насколько гиперпараметры обучения влияют на результат

**Варианты архитектуры сверточной сети:**
Вариант на ваш выбор - напишите его в конфу. Не более двух человек на один вариант
3. MobileNet v2

**Варианты оптимизатора:**
Для дополнительных баллов, только один вариант на человека
1. Sharpness Aware Optimization (+1.5 балл)

## Имплементация сети на Pytorch
1. Не забывать про использоватие блоков nn.Module, nn.Sequential, nn.ModuleList
2. Использовать материалы из предыдущих семинаров
"""

# В качестве датасета возьмем MNIST с 10 классами

import torch.nn as nn
from torchvision.models import mobilenet_v2
from torchvision.models.mobilenetv2 import InvertedResidual
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score
from torchvision import transforms
import os
import gzip
import numpy as np
import urllib.request
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.loggers import TensorBoardLogger

# особенность mobilenetv2 - InvertedResidual - сначала выполняется уменьшение размерности, а затем увеличение (в ResNet наоборот)

class SELayer(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # глобальное усредненное значение для каждого канала (GAP)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction),  # каналы уменьшаются в reduction раз
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel),  # возвращение исходного значения
        )

    def forward(self, x):
        b, c, _, _ = x.size()  # [batch_size, channels, height, width]
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        y = torch.clamp(y, 0, 1)  # ограничивает значения y [0, 1]. y - весовой коэффициент для каждого канала
        return x * y


class MyMobileNetV2(nn.Module):
    def __init__(self, num_classes=10, pretrained=True, learning_rate=0.001, rho=0.05, reduction=4):
        """
        Инициализация класса MobileNetV2 из торча

        Args:
            num_classes (int, optional): Количество классов, которые будет классифицировать модель. Defaults to 10.
            pretrained (bool, optional): Предобученная ли модель. Defaults to True.
            learning_rate (float, optional): _description_. Defaults to 0.001.
            rho (float, optional): Величина смещения, по формуле: Δw = rho * (grad / ||grad||). ||grad|| - норма градиента. Defaults to 0.05.
            reduction (int, optional): Степень сокращения количества нейронов в скрытом слое SE-блока. Сужает внимание перед проведение адаптации.
        """
        super().__init__()  # наследуем параметры, которые лежат в Mobilenev2 в pytorch

        self.mobilenet = mobilenet_v2(pretrained=pretrained)

        for i, layer in enumerate(self.mobilenet.features):
            if isinstance(layer, InvertedResidual):
                # Получаем последний слой в conv
                last_layer = layer.conv[-1]

                # Проверяем, является ли последний слой сверткой
                if isinstance(last_layer, nn.Conv2d):
                    in_channels = last_layer.out_channels  # количество каналов после свертки
                    self.mobilenet.features[i] = nn.Sequential(
                        layer,
                        SELayer(in_channels, reduction=reduction)
                    )

        # Заменяем последний слой, задаем новый num_classes
        in_features = self.mobilenet.classifier[1].in_features
        self.mobilenet.classifier = nn.Sequential(  # два последовательных слоя. В качестве регуляризации добавим словй nn.Dropout
            nn.Dropout(0.4),
            nn.Linear(in_features, num_classes)
        )

        # Параметры оптимизации
        self.learning_rate = learning_rate
        self.rho = rho  # гиперпараметр для SAO

    def forward(self, x):
        # Проход через MobileNetV2
        x = self.mobilenet(x)
        return x


    def sharpness_aware_step(self, optimizer, loss_fn, X, y):
        """
        Выполняет один шаг оптимизации с использованием Sharpness-Aware Optimization.
        """
        # Вычисление начальных градиентов
        optimizer.zero_grad()
        loss = loss_fn(self(X), y)
        loss.backward()

        # Сохранение копий текущих параметров
        original_params = [p.clone() for p in self.parameters()]

        # Искажение параметров на величину rho для SAO
        with torch.no_grad():
          for p in self.parameters():
              if p.grad is not None:
                  p.data += self.rho * p.grad / (torch.norm(p.grad) + 1e-8)

        # Второй шаг: вычисление и применение градиентов на искаженных параметрах
        optimizer.zero_grad()
        perturbed_loss = loss_fn(self(X), y)
        perturbed_loss.backward()

        # Восстановление исходных параметров и обновление оптимизатора
        for p, original_p in zip(self.parameters(), original_params):
            p.data = original_p.data

        optimizer.step()

        return perturbed_loss.item()

    def get_optimizer(self):
        # Оптимизатор, который будет использоваться для SAO
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)



class ConvModelPL(pl.LightningModule):
  def __init__(self, model, lr, weight_decay, rho=0.05):
    super().__init__()
    self.model = model
    self.lr = lr
    self.weight_decay = weight_decay
    self.rho = rho
    self.validation_outputs = []


  def forward(self, x):
      return self.model(x)


  def training_step(self, batch, batch_idx):
        x, y = batch
        loss_fn = torch.nn.CrossEntropyLoss()
        loss = loss_fn(self(x), y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss



  def validation_step(self, batch, batch_idx):
    # соответсвенно, здесь выполняется шаг валидации
    # тоже нужно сделать forward модели и подсчитать лосс
    # но кроме этого - вычислить метрику

    x, y = batch

    loss_fn = torch.nn.CrossEntropyLoss()
    logits = self.model(x)

    loss = loss_fn(logits, y)

    preds = torch.argmax(logits, dim=1).cpu().numpy()
    target = y.cpu().numpy()
    metric = accuracy_score(target, preds)

    self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    self.log("val_accuracy", metric, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    # потом мы снова этот вывод усредним, сохраняем
    self.validation_outputs.append(logits)


  def on_validation_epoch_end(self):
      # усредним накопленные выходы
      if self.validation_outputs:
          # Конкатенируем все выходы валидации
          all_outputs = torch.cat(self.validation_outputs)
          avg_accuracy = all_outputs.mean()
          self.log("val_epoch_acc", avg_accuracy, prog_bar=True, logger=True)

      # Очищаем список для следующей эпохи
      self.validation_outputs.clear()


  def configure_optimizers(self):
      # здесь мы настраиваем оптимизатор
      base_optimizer = torch.optim.AdamW
      optimizer = SAO(self.model.parameters(), base_optimizer, lr=self.lr, alpha=self.rho)

      return optimizer

def load_model():
    model = MyMobileNetV2()
    model_pl = ConvModelPL(model, lr=1e-4, weight_decay=1e-6)
    return model_pl


def load_mnist(flatten=False):
    """Загружает датасет MNIST с GitHub репозитория и кэширует его локально."""

    # Ссылки на файлы MNIST в репозитории GitHub -
    base_url = "https://raw.githubusercontent.com/golbin/TensorFlow-MNIST/master/mnist/data/"
    filenames = {
        "train_images": "train-images-idx3-ubyte.gz",
        "train_labels": "train-labels-idx1-ubyte.gz",
        "test_images": "t10k-images-idx3-ubyte.gz",
        "test_labels": "t10k-labels-idx1-ubyte.gz"
    }

    def download(filename):
        url = base_url + filename
        print(f"Downloading {filename} from {url}")
        urllib.request.urlretrieve(url, filename)

    # Функции для загрузки изображений и меток
    def load_mnist_images(filename):
      if not os.path.exists(filename):
          download(filename)
      with gzip.open(filename, 'rb') as f:
          data = np.frombuffer(f.read(), np.uint8, offset=16)

      # (-1, 28, 28) для изображений в оттенках серого
      data = data.reshape(-1, 28, 28)

      # преобразуем в формат цветного
      data_rgb = np.stack((data,) * 3, axis=1)  # Дублируем канал

      return data_rgb / np.float32(256)

    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        return data

    X_train = load_mnist_images(filenames["train_images"])
    y_train = load_mnist_labels(filenames["train_labels"])
    X_test = load_mnist_images(filenames["test_images"])
    y_test = load_mnist_labels(filenames["test_labels"])

    indices_train = np.random.permutation(len(X_train))
    X_train, y_train = X_train[indices_train], y_train[indices_train]

    X_val = X_train[10000:15000]
    y_val = y_train[10000:15000]

    X_train = X_train[:10000]
    y_train = y_train[:10000]

    if flatten:
        X_train = X_train.reshape([X_train.shape[0], -1])
        X_val = X_val.reshape([X_val.shape[0], -1])
        X_test = X_test.reshape([X_test.shape[0], -1])

    return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
def run_training():
    X_train, y_train, X_val, y_val, _, _ = load_mnist(flatten=False)

    train_set = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long())
    val_set = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).long())
    train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=4)

    logger = TensorBoardLogger("outputs", name="logs")

    model = MyMobileNetV2(num_classes=10, pretrained=True)
    model_pl = ConvModelPL(model, lr=1e-4, weight_decay=1e-6, rho=0.05)

    trainer = pl.Trainer(max_epochs=20, devices=1, accelerator='cuda', logger=logger)
    trainer.fit(model_pl, train_loader, val_loader)


if __name__ == '__main__':
    run_training()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_su

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


## Вывод после первой итерации:

250/250 [00:09<00:00, 26.69it/s, v_num=11, train_loss_step=1.630, val_loss=1.740, val_accuracy=0.440, val_epoch_acc=-0.445, train_loss_epoch=1.880]

Результаты плохие, поменяем параметры обучения

1. усложним аугментацию данных
2. увеличим их количество
3. обучаем всю модель, а не только последний классификационный слой
4. настроим гиперпараметры
5. настроем sheduler чтобы динамически изменять скорость обучения learning rate
6. увеличим количество эпох



In [ ]:
def run_training():
    # Применяем аугментации
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    X_train, y_train, X_val, y_val, _, _ = load_mnist(flatten=False)

    train_set = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long())
    val_set = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).long())

    # увеличение batch_size
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32)

    logger = TensorBoardLogger("outputs", name="logs")

    model = MyMobileNetV2(num_classes=10, pretrained=True)
    model_pl = ConvModelPL(model, lr=5e-4, weight_decay=1e-4, rho=0.05)


    # увеличение эпох
    trainer = pl.Trainer(max_epochs=50, devices=1, accelerator='cuda', logger=logger)
    trainer.fit(model_pl, train_loader, val_loader)


if __name__ == '__main__':
    run_training()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_su

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Как результат видим, что применение всех этих техник немного помогло улучшить качество обучения модели. Однако для получения лучших результатов, необходимо ещедонастраивать модель

## Вывод после второй итерации

p.s. потом я поняла, что проблема в датасете mnist-a, т.к. он весь в оттенках серого, а я загружаю его в RGB)